In [ ]:
!pip install yfinance
!pip install nbformat
!pip install statsmodels

### collect real-time stock market data using the yfinance API:

In [ ]:
import pandas as pd
import yfinance as yf 
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months = 3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers , names = ['Ticker', 'Date'])
print(df.head())


In [ ]:
#In this dataset, the Date column is the index column in the DataFrame. We need to reset the index before moving forward:
df = df.reset_index()
print(df.head())

In [13]:
# the performance in the stock market of all the companies:
import plotly.express as px 
fig = px.line(df, x='Date', y='Close', color='Ticker' , title='Stock Market Performance for the Last 3 Months')
fig.show()

In [14]:
#the faceted area chart, which makes it easy to compare the performance of different companies and identify similarities or differences in their stock price movements
fig = px.area(df, x='Date', y='Close', color = 'Ticker', facet_col='Ticker' , labels = {'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'} , title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

### analyze moving averages, which provide a useful way to identify trends and patterns in each company’s stock price movements over a period of time:

In [ ]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window = 10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window = 20).mean().reset_index(0, drop=True)

for ticker ,group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

In [16]:
#visualize the moving averages of all companies:
for ticker , group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], title=f'{ticker} Moving Averages')
    fig.show()

In [17]:
### Volatility is a measure of how much and how often the stock price or market fluctuates over a given period of time. Here’s how to visualize the volatility of all companies:

In [18]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window = 10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', color='Ticker', title='Volatility of Apple, Microsoft, Netflix, and Google')
fig.show()

### Now let’s analyze the correlation between the stock prices of Apple and Microsoft:

In [19]:
apple = df.loc[df['Ticker'] == 'AAPL' , ['Date', 'Close']].rename(columns = {'Close':'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT' , ['Date', 'Close']].rename(columns = {'Close':'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

fig = px.scatter(df_corr, x='AAPL', y='MSFT', trendline='ols' , title='Correlation between Apple and Microsoft')
fig.show()